In [14]:
import pandas as pd
import preprocessing
from importlib import reload

In [2]:
target_folder = 'Dataset'

In [3]:
train = pd.read_csv(target_folder + '/train.csv')
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
test = pd.read_csv(target_folder + '/test.csv')
test

In [6]:
sumbission_example = pd.read_csv(target_folder + '/sample_submission.csv')
sumbission_example

In [10]:
train['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [25]:
reload(preprocessing)
print(train['comment_text'][0])
#preprocessing.preprocess((train['comment_text'][0],))
preprocessing.preprocess(train['comment_text'][0])

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


TextBlob("Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27")

In [46]:
from textblob import TextBlob
from nltk.tokenize import TabTokenizer, BlanklineTokenizer
TextBlob(train['comment_text'][0]).tags

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michael/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('Explanation', 'NN'),
 ('Why', 'WRB'),
 ('the', 'DT'),
 ('edits', 'NNS'),
 ('made', 'VBN'),
 ('under', 'IN'),
 ('my', 'PRP$'),
 ('username', 'JJ'),
 ('Hardcore', 'NNP'),
 ('Metallica', 'NNP'),
 ('Fan', 'NNP'),
 ('were', 'VBD'),
 ('reverted', 'VBN'),
 ('They', 'PRP'),
 ('were', 'VBD'),
 ("n't", 'RB'),
 ('vandalisms', 'JJ'),
 ('just', 'RB'),
 ('closure', 'NN'),
 ('on', 'IN'),
 ('some', 'DT'),
 ('GAs', 'NNP'),
 ('after', 'IN'),
 ('I', 'PRP'),
 ('voted', 'VBD'),
 ('at', 'IN'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Dolls', 'NNP'),
 ('FAC', 'NNP'),
 ('And', 'CC'),
 ('please', 'VB'),
 ('do', 'VBP'),
 ("n't", 'RB'),
 ('remove', 'VB'),
 ('the', 'DT'),
 ('template', 'NN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('talk', 'NN'),
 ('page', 'NN'),
 ('since', 'IN'),
 ('I', 'PRP'),
 ("'m", 'VBP'),
 ('retired', 'JJ'),
 ('now.89.205.38.27', 'NN')]